In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
# import json
# from scipy import stats
import time

def runtime(s):
    hours, remainder = divmod(s, 3600)
    minutes, seconds = divmod(remainder, 60)
    print('Runtime: {:02}:{:02}:{:02}'.format(int(hours), int(minutes), int(seconds)))

def create_SubjCode(userNr, recordingNr):
    # SubjCode = userNr + recordingNr
    # pvz. SubjCode = 10002
    if (userNr < 1000):  # duomenys mit2zive 
        return userNr
    else:  # duomenys zive      
        str_code = str(userNr) + str(recordingNr)
        SubjCode = int(str_code)
        return SubjCode

def get_SubjCode(idx, all_beats_attr):
    row = all_beats_attr.loc[idx]
    SubjCode = create_SubjCode(row['userNr'],  row['recordingNr'])
    return SubjCode


def get_seq_start_end(signal_length,i_sample,window_left_side,window_right_side):
    # Nustatome išskiriamos EKG sekos pradžią ir pabaigą
    seq_start = i_sample - window_left_side
    seq_end = i_sample + window_right_side
    if (seq_start < 0 or seq_end > signal_length):
        # print("\nseq_start: ", seq_start, " seq_end: ", seq_end)
        return (None,None)
    else:    
        return (seq_start, seq_end)



def read_seq_RR(rec_dir, all_beats_attr, idx, wl_side, wr_side):
# Nuskaito ir pateikia EKG seką apie R dantelį: wl_side - iš kairės pusės, wr_side - iš dešinės pusės,
# klasės numerį: 0, 1, 2.
# Taip pat pateikia RRl - EKG reikšmių skaičių nuo R dantelio iki prieš tai buvusio R dantelio,
# ir RRr - reikšmių skaičių nuo R dantelio iki sekančio R dantelio

    row = all_beats_attr.loc[idx]
    SubjCode = get_SubjCode(idx, all_beats_attr) 

    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    signal = np.load(file_path, mmap_mode='r')    
    signal_length = signal.shape[0]
    (seq_start, seq_end)  = get_seq_start_end(signal_length, row['sample'], wl_side, wr_side)

    RRl = row['RRl']
    RRr = row['RRr']

    # Užfiksuojame per ilgas sekas įrašo pradžioje ir pabaigoje
    if (seq_start == None or seq_end == None):
        raise Exception(f"Klaida! {idx}: Sekos rėžiai už EKG įrašo ribų") 
    else:    
        seq = signal[seq_start:seq_end]
        sample = row['sample']
        label = row['label']


    return seq, sample, label, RRl, RRr


def read_seq_RR_arr(rec_dir, all_beats_attr, idx, wl_side, wr_side, nl_steps=0, nr_steps=0):
# Nuskaito ir pateikia EKG seką apie R dantelį seq: reikšmiu kiekis wl_side - iš kairės pusės, 
# reikšmiu kiekis wr_side - iš dešinės pusės, R dantelio vietą EKG įraše sample,
# ir atitinkamo pūpsnio klasės numerį label: 0, 1, 2.
# Taip pat pateikia seką RRl_arr iš nl_steps RR reikšmių tarp iš eilės einančių R dantelių į kairę nuo einamo R dantelio 
# ir seką RRr_arr nr_steps RR reikšmių tarp iš eilės einančių R dantelių į dešinę nuo einamo R dantelio dantelio.
# Seka iš kairės RRl_arr prasideda nuo tolimiausio nuo R dantelio atskaitymo ir jai pasibaigus,
# toliau ją pratesia RRl_arr. 

    row = all_beats_attr.loc[idx]
    SubjCode = get_SubjCode(idx, all_beats_attr)

    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    
    signal = np.load(file_path, mmap_mode='r')     
    signal_length = signal.shape[0]
    (seq_start, seq_end)  = get_seq_start_end(signal_length, row['sample'], wl_side, wr_side)

# **************************** Tikrinimai ******************************************************************
    # Tikriname, ar sekos langas neišeina už įrašo ribų
    if (seq_start == None or seq_end == None):
        raise Exception(f"Klaida! {idx}: Sekos lango rėžiai už EKG įrašo {SubjCode} ribų.") 
        # Reikia mažinti wl_side ar wr_side, arba koreguoti idx ribas 
    else:    
        seq = signal[seq_start:seq_end]
        sample = row['sample']
        label = row['label']

    # Tikriname, ar skaičiuodami RR neišeisime už all_beats_attr ribų
    if (idx + nr_steps) >= len(all_beats_attr):
        txt = f"Klaida 1! idx, nl_steps: {idx}, {nr_steps} Skaičiuojant RR viršijama pūpsnių atributo masyvo riba." 
        raise Exception(txt)  
        # Reikia mažinti nr_steps arba koreguoti viršutinę idx ribą
    
    if ((idx - nl_steps) < 0):
        txt = f"Klaida 2! idx, nl_steps: {idx}, {nl_steps} Skaičiuojant RR išeinama už pūpsnių atributo masyvo ribų."
        raise Exception(txt)  
        # Reikia mažinti nl_steps arba didinti apatinę idx ribą

    # Tikriname, ar skaičiuodami RR neišeisime už EKG įrašo SubjCode ribų
    SubjCodeRight = get_SubjCode(idx + nr_steps, all_beats_attr)
    if (SubjCodeRight != SubjCode):
        txt = f"Klaida 3! idx, nl_steps: {idx}, {nr_steps} Skaičiuojant RR viršijama EKG įrašo {SubjCode} viršutinė riba."
        raise Exception(txt)  
        # Reikia mažinti nr_steps arba mažinti max idx 

    SubjCodeLeft = get_SubjCode(idx - nl_steps, all_beats_attr)
    if (SubjCodeLeft != SubjCode):
        txt = f"Klaida 4! idx, nl_steps: {idx}, {nl_steps} Skaičiuojant RR išeinama už EKG įrašo {SubjCode} apatinės ribos."
        raise Exception(txt)  
        # Reikia koreguoti nl_steps arba koreguoti apatinę idx ribas 

# **************************** Tikrinimų pabaiga ******************************************************************

    # Suformuojame RR sekas kairėje ir dešinėje idx atžvilgiu
    if (nl_steps != 0):
        RRl_arr = np.zeros(shape=(nl_steps), dtype=int)
        for i in range(nl_steps):
            RRl_arr[nl_steps-i-1] = all_beats_attr.loc[idx-i, 'sample'] - all_beats_attr.loc[idx-i-1, 'sample']
    else:    
        RRl_arr = None

    if (nr_steps != 0):
        RRr_arr = np.zeros(shape=(nr_steps), dtype=int)
        for i in range(nr_steps):
            RRr_arr[i] = all_beats_attr.loc[idx+i+1, 'sample'] - all_beats_attr.loc[idx+i, 'sample']
    else:
        RRr_arr = None        
    
    return seq, sample, label, RRl_arr, RRr_arr

def create_set_senas(rec_dir, all_beats_attr, ind_lst, window_left_side, window_right_side):
    # Panaudojant sekų atributų freimą, sukuriamas užduoto ilgio sekų ir klasių numerių masyvai, 
    # tinkami klasifikatoriaus mokymui ir tikslumo vertinimui
    
    seq_length = window_left_side + window_right_side

    # Suformuojame failus pildymui
    set_len = len(ind_lst)
    X = np.empty((set_len, seq_length))
    y = np.empty((set_len), dtype=int)

    # Pildymas
    count = 0
    for idx in ind_lst:
        seq_1d, sample, label, RRl, RRr = read_seq_RR(rec_dir, all_beats_attr, idx, window_left_side, window_right_side)
        if (label != None):
            X[count,:] = seq_1d 
            y[count] = label
            count +=1
        else:
            print("Klaida!", idx)    

    # Koreguojame ilgius 
    X_set = np.resize(X, (count, seq_length))
    y_set = np.resize(y, (count))
    return X_set, y_set

def create_set_naujas(rec_dir, all_beats_attr, ind_lst, window_left_side, window_right_side, nl_steps, nr_steps):
    # Panaudojant sekų atributų freimą, sukuriamas užduoto ilgio sekų ir klasių numerių masyvai, 
    # tinkami klasifikatoriaus mokymui ir tikslumo vertinimui
    
    seq_length = window_left_side + window_right_side

    # Suformuojame failus pildymui
    set_len = len(ind_lst)
    X = np.empty((set_len, seq_length))
    y = np.empty((set_len), dtype=int)

    # Pildymas
    count = 0
    for idx in ind_lst:
        seq_1d, sample, label, RRl_arr, RRr_arr = read_seq_RR_arr(rec_dir, all_beats_attr, idx,
                                             window_left_side, window_right_side, nl_steps, nr_steps)
        if (label != None):
            X[count,:] = seq_1d 
            y[count] = label
            count +=1
        else:
            print("Klaida!", idx)    

    # Koreguojame ilgius 
    X_set = np.resize(X, (count, seq_length))
    y_set = np.resize(y, (count))
    return X_set, y_set


# TESTAVIMAS

def print_stats(stats, arr_values):
    print(f'min: {stats.minmax[0]:.5f}, max: {stats.minmax[1]:.4f}  mean: {stats.mean:.5f}  standard: {np.std(arr_values):.5f}')
    print(f'variance: {stats.variance:.5f}  skewness: {stats.skewness:.5f}  kurtosis: {stats.kurtosis:.5f}')


import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'


# Vietinės talpyklos aplankas
db_folder = 'DUOM_VU'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)
rec_dir = Path(db_path,'records_npy')

print("Duomenys skaitomi iš:", rec_dir)
print("\n")

# Testas sekų skaitymui su read_seq_RR
print("\nTestas sekų skaitymui su read_seq_RR")

# *********************** parametrai ********************************
wl_side, wr_side = 5,5
nl_steps, nr_steps = 7,3
idx_min, idx_max = 2280,2280 
# idx_min, idx_max = 322990,322993 
print(f"wl_side = {wl_side} wr_side = {wl_side}")
print(f"nl_steps = {nl_steps} nr_steps = {nr_steps}")
print(f"idx_min = {idx_min} idx_max = {idx_max}")
print("\n")
# *********************************************************************

idxs = [i for i in range(idx_min,idx_max+1)]

# Nuskaitome pūpsnių atributų failą
file_path = Path(rec_dir, 'all_beats_attr_z.csv')
all_beats_attr = pd.read_csv(file_path, index_col=0)

                    # Testas klaidų tikrinimui
#Testas su nauja funkcija
for idx in idxs:
    print("\n")
    seq_1d, sample, label, RRl_arr, RRr_arr = read_seq_RR_arr(rec_dir, all_beats_attr, idx,
     wl_side, wr_side, nl_steps, nr_steps)
    SubjCode = get_SubjCode(idx, all_beats_attr)
    print(f"SubjCode = {SubjCode} idx = {idx}  seq_1d.shape = {seq_1d.shape}  sample = {sample}  label = {label}")
    print(f"RRl_arr: {RRl_arr}")
    print(f"RRr_arr: {RRr_arr}")
    print('seq_1d:', seq_1d)

    # statistics = stats.describe(seq_1d)
    # print_stats(statistics, seq_1d)

# Testas su sena funkcija
    seq_1d_old, sample, label, RRl, RRr = read_seq_RR(rec_dir, all_beats_attr, idx, wl_side, wr_side)
    print(f"idx = {idx}  seq_1d_old.shape = {seq_1d_old.shape}  sample = {sample}  label = {label}")
    print(f"RRl: {RRl}")
    print(f"RRr: {RRr}")
    print('seq_1d_old:', seq_1d_old)

                    # Testas masyvo formavimui

# Suformuojame  imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

# Imties pacientai
DS1 = [10020, 10021, 10051]

# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 3

index_beats_attr = all_beats_attr.index
ind_lst = []
for userNr in DS1:
    selected_ind = index_beats_attr[all_beats_attr['userNr']==userNr]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    ind_lst.extend(selected_ind.to_list())


# Testas su sena funkcija
# Suformuojame mokymo sekų ir jų klasių numerių masyvus
start_time = time.time()
X, y = create_set_senas(rec_dir, all_beats_attr, ind_lst, wl_side, wr_side)
print("\nImties dimensijos")
print(f"{X.shape} {y.shape}")

end_time = time.time()
runtime(end_time-start_time)

nl_steps, nr_steps = 3,3

# Testas su nauja funkcija
# Suformuojame mokymo sekų ir jų klasių numerių masyvus
start_time = time.time()
X, y = create_set_naujas(rec_dir, all_beats_attr, ind_lst, wl_side, wr_side,nl_steps, nr_steps)
print("\nImties dimensijos")
print(f"{X.shape} {y.shape}")

end_time = time.time()
runtime(end_time-start_time)


OS in my system :  linux
Duomenys skaitomi iš: /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy



Testas sekų skaitymui su read_seq_RR
wl_side = 5 wr_side = 5
nl_steps = 7 nr_steps = 3
idx_min = 2280 idx_max = 2280




FileNotFoundError: [Errno 2] No such file or directory: '/home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy/all_beats_attr_joint.csv'